우리가 아직 죄인 되었을 때에 그리스도께서 우리를 위하여 죽으심으로 하나님께서 우리에 대한 자기의 사랑을 확증하셨느니라.	(롬5:8)
<center><img src="https://github.com/idebtor/KMOOC-ML/blob/master/ipynb/images/MLwithPython.png?raw=true" width=1000></center>

__NOTE:__ The following materials have been compiled and adapted from the numerous sources including my own. Please help me to keep this tutorial up-to-date by reporting any issues or questions. Send any comments or criticisms to `idebtor@gmail.com` Your assistances and comments will be appreciated.

# 제 11-1 강: 다층 신경망의 행렬 모델링 

## 학습 목표
- 미분의 연쇄법칙을 학습한다.
- 오차함수의 행렬 표기에서 미분하는 방법을 학습한다.
- 다층 인공 신경망의 행렬 모델을 학습한다. 

## 학습 내용
- 미분의 연쇄법칙
- 오차함수의 행렬 미분
- 다층 인공 신경망 행렬 모델


## 1. 다층 신경망의 구조

우리는 지금까지 다층 신경망의 신호처리를 공부했습니다.  수학적 모델이 참 복잡했지만, 이제 여기서 다층 신경망을 일련의 행렬의 곱셈으로 다시 한번 모델링을 해보겠습니다.   신경망의 신호처리를 행렬로 모델링하는 이유는 파이썬 넘피를 사용하여 간편한 코딩으로 더 빠르게 연산이 가능하게 할 뿐만 아니라 GPU/TPU 등 병렬처리 하드웨어를 활용할 수 있는 잇점도 있기 때문입니다.  

다음의 신경망을 생각해봅시다. 
<center><img src="https://github.com/idebtor/KMOOC-ML/blob/master/ipynb/images/ch10/multi-layerNN-1.PNG?raw=true" width="500">
<center>그림 1: 다층 신경망의 구조</center>

그림 1은 다층 신경망의 구조를 보여줍니다.  입력층과 출력층은 신경망의 첫 단과 끝 단을 지정한 것이므로, 언제나 각각 한 층씩이며, 은닉층은 여러 개가 있을 수 있습니다.  은닉층이 많아지면 심화학습 즉 딥 러닝$^{deep \ learning}$을 할 수 있는 신경망이 되는 것입니다.  어떤 문헌들은 입력층은 뉴론으로 구성되어 있지 않으며, 입력 값을 단순히 전달하는 역할만 하므로, 하나의 층으로 간주하지 않기도 합니다. 

각 층의 사이에는 신경망으로 하여금 학습 능력을 갖게 하는 가중치 W가 존재합니다.   가중치는 행렬로 표기합니다. 가중치 행렬의 크기는 가중치 행렬의 앞과 뒤에 있는 층의 노드 수가 결정합니다.  물론, $W_{ij}$ 혹은 $W_{ij}^T$ 표기법을 사용할 것인지에 따라 행렬의 형상$^{shape}$은 달라지지만 그 내용은 같습니다. 

그러므로, 그림 1에서 입력층과 출력층 행렬의 크기는 기계학습을 위하여 준비된 자료의 특성에 따라 그 크기가 각각 결정이 됩니다.  은닉층의 크기 즉 노드의 수와 몇 개의 은닉층의 깊이는 자료의 특성과 목적에 따라 결정되어야 합니다.  


## 2. 다층 신경망의 입력과 출력

그림 2는 다층 신경망 각 층의 입력과 출력을 보여줍니다. 

<center><img src="https://github.com/idebtor/KMOOC-ML/blob/master/ipynb/images/ch10/multi-layerNN-2.PNG?raw=true" width="600">
<center>그림 2: 다층 신경망의 입력과 출력</center>

여기서 사용하는 입력과 출력 기호의 윗첨자는 입력층 0를 시작으로 층의 일련 번호입니다.  X는 기계학습을 위한 입력 자료입니다.  입력층에서는 X에 대해 아무런 연산이 없이 출력하므로,  입력층의 출력 $A^{[0]} = X$가 됩니다.  
일반적으로, Z와 A는 각각 층의 입력과 출력을 나타냅니다.  예를 들면, $Z^{[1]}$은 신경망 1층의 입력이며, $A^{[2]}$은 신경망 2층의 출력을 나타냅니다. 물론 신경망 2층이 마지막 층이므로, 이 출력은 곧 전체 신경망의 최종 출력이 되며, 이는 $\hat{y}$로 표기합니다.  

## 3. 순전파 모델링 

다층 신경망에 대한 입출력을 바탕으로 입력 신호의 순방향으로 전파되면서 연속적으로 일어나는 연산이 일어나는 것을 관찰할 수 있으며, 이를 __순전파__(feedforward propagation)라고 부릅니다.   순전파 신경망에서 일어나는 연산을 행렬로 표기하면 다음과 같습니다. 

<center><img src="https://github.com/idebtor/KMOOC-ML/blob/master/ipynb/images/ch10/multi-layerNN-3.PNG?raw=true" width="600">
<center>그림 3: 다층 신경망의 입출력과 연산</center>

함수 $g(\cdot)$은 각층의 활성화 함수를 표시합니다.  층마다 각각 서로 다른 활성화 함수를 사용할 수도 있습니다.  

입력층을 제외하고, 각 층의 출력은 각 층의 입력에 활성화 함수를 적용한 값입니다. 입력층을 제외하고, 각 층의 입력은 가중치 행렬과 그 앞층의 출력값의 곱(내적)인 것을 관찰할 수 있습니다.

입력층에서는 입력 벡터 X을 연산 없이 출력하므로 $X = A^{[0]}$이 성립됩니다. 은닉층에서는 $A^{[0]}$와 은닉층에서의 가중치 $W^{[1]}$을 곱(내적)하여 $Z^{[1]}$을 만들어냅니다. 이 입력값은 활성화 함수에 적용되어 $A^{[1]} = g(Z^{[1]})$을 만들어냅니다. 즉, 은닉층에서의 연산은 다음과 같습니다.

\begin{align} Z^{[1]} = W^{[1]}A^{[0]}\end{align}
\begin{align} A^{[1]} = g(Z^{[1]}) \end{align}

이와 같은 연산은 은닉층에서 출력층 사이에도 똑같이 적용됩니다. 은닉층에서 입력받은 $A^{[1]}$을 2층 가중치 $W^{[2]}$와 곱(내적)하여$Z^{[2]}$을 계산하고 $Z^{[2]}$에 활성화 함수를 적용하여 $A^{[2]}$을 만들어냅니다. 이 결과값 $A^{[2]}$은 최종 출력값인 $\hat{y}$이 됩니다. 이를 수식으로 표현하면 다음과 같습니다.

\begin{align} Z^{[2]} = W^{[2]}A^{[1]}\end{align}
\begin{align} A^{[2]} = g(Z^{[2]}) \end{align}
\begin{align} \hat{y} = A^{[2]} \end{align}

이를 일반화하면, 다음과 같은 공식이 성립이 됩니다. 

\begin{align}   Z^{[l]} = W^{[l]}\cdot A^{[l-1]} \end{align}
\begin{align}   A^{[l]} = g(Z^{[l]}) \end{align}

## 4. 역전파 모델링 
 

다층 신경망 순전파의 결과는 출력층에서 출력값 $A$ 즉 $\hat{y}$으로 나타납니다.  다음 단계는 출력값 $\hat{y}$과 주어진 클래스 레이블 $y$과 비교하여 오차 $E$를 계산하여, 오차를 줄이는 작업을 작업해야 하는데, 이 오차를 줄이는 방법 중에 하나가 경사하강법입니다. 오차함수를 미분하여 오차함수의 기울기를 구합니다. 기울기가 양수이면, 현재의 가중치를 줄이고, 기울기가 음수이면 가중치를 늘리는 작업을 반복하면서 오차함수의 최소값으로 수렴해가는 방법입니다.  출력층에 나타난 오차를 또한 그 앞에 층으로 가중치에 비례 배분하여 순전파와는 반대 방향으로 전파하는 과정을 __역전파__(backpropagation)라고 부릅니다.  
오차 함수를 미분한 값 즉 기울기와 오차를 이용하여 가중치를 조정해나가는 과정을 신경망의 역전파라고 합니다. 출력층에서 가까운 가중치부터 조정하고, 그 다음에는 역순으로 계속해서 첫번째 은닉층에 사용한 가중치를 조정할 때까지 역전파를 계속해야 합니다.  

다음 그림 4에서 점선으로 표시된 부분이 역전파 과정을 보여줍니다.  

<center><img src="https://github.com/idebtor/KMOOC-ML/blob/master/ipynb/images/ch9/multi-layerNN-4.PNG?raw=true" width="700">
<center>그림 4: 다층 신경망의 역전파</center>

## 학습 정리

- 미분에서 연쇄법칙이 무엇인지 이해하기
- 오차함수의 행렬로 미분하기
- 다층 인공 신경망의 행렬 모델을 학습하기

    
    
    

-----
_For God so loved the world that he gave his one and only Son, that whoever believes in him shall not perish but have eternal life. John3:16_